In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb

In [2]:
import pandas as pd


train = np.asarray(pd.read_csv('/Users/michaelhorrell/Downloads/adult.data'))
test = np.asarray(pd.read_csv('/Users/michaelhorrell/Downloads/adult.test'))

In [3]:
ntrain = train.shape[0]
full = np.concatenate([train,test],axis=0)

c0 = full[:,0].astype(float).reshape([-1,1])
c1 = np.asarray(pd.get_dummies(full[:,1]))
c2 = full[:,2].astype(float).reshape([-1,1])
c3 = np.asarray(pd.get_dummies(full[:,3]))
c4 = full[:,4].astype(float).reshape([-1,1])
c5 = np.asarray(pd.get_dummies(full[:,5]))
c6 = np.asarray(pd.get_dummies(full[:,6]))
c7 = np.asarray(pd.get_dummies(full[:,7]))
c8 = np.asarray(pd.get_dummies(full[:,8]))
c9 = np.asarray(pd.get_dummies(full[:,9]))
c10 = full[:,10].astype(float).reshape([-1,1])
c11 = full[:,11].astype(float).reshape([-1,1])
c12 = full[:,12].astype(float).reshape([-1,1])
c13 = np.asarray(pd.get_dummies(full[:,13]))

label = np.zeros([full.shape[0],1])
label[full[:,14] == ' >50K',0] = 1
label[full[:,14] == ' >50K.',0] = 1

XFULL = np.concatenate([c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13],axis=1)
YFULL = label

xtrain = XFULL[0:ntrain,:]
ytrain = label[0:ntrain,0]

xtest = XFULL[ntrain:,:]
ytest = label[ntrain:,0]

train_data = lgb.Dataset(xtrain, label=ytrain)
test_data = lgb.Dataset(xtest, label=ytest)

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = lgb.train(param,
                      train_data,
                      num_boost_round=100,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['binary_error'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'objective': hp.choice('objective',['binary']),
    'metric':hp.choice('metric',['binary_error']),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.633397511431324, 'feature_fraction': 0.5790808370797315, 'lambda_l1': 2.79860335652886e-07, 'lambda_l2': 1.5238304007383441e-05, 'learning_rate': 0.008415710548353484, 'metric': 'binary_error', 'min_data_in_leaf': 19.0, 'min_sum_hessian_in_leaf': 0.0004623879351153157, 'num_leaves': 1.0, 'objective': 'binary'}
[1]	valid_0's binary_error: 0.236241                   
[2]	valid_0's binary_error: 0.236241                   
[3]	valid_0's binary_error: 0.236241                   
[4]	valid_0's binary_error: 0.236241                   
[5]	valid_0's binary_error: 0.236241                   
[6]	valid_0's binary_error: 0.236241                   
[7]	valid_0's binary_error: 0.236241                   
[8]	valid_0's binary_error: 0.236241                   
[9]	valid_0's binary_error: 0.236241                   
[10]	valid_0's binary_error: 0.236241                  
[11]	valid_0's binary_error: 0.236241                  
[12]	valid_0's binary_error: 0.236241            

[23]	valid_0's binary_error: 0.142875                                             
[24]	valid_0's binary_error: 0.141646                                             
[25]	valid_0's binary_error: 0.140602                                             
[26]	valid_0's binary_error: 0.139926                                             
[27]	valid_0's binary_error: 0.138575                                             
[28]	valid_0's binary_error: 0.138022                                             
[29]	valid_0's binary_error: 0.137654                                             
[30]	valid_0's binary_error: 0.137101                                             
[31]	valid_0's binary_error: 0.136179                                             
[32]	valid_0's binary_error: 0.136732                                             
[33]	valid_0's binary_error: 0.136057                                             
[34]	valid_0's binary_error: 0.134951                                             
[35]

[17]	valid_0's binary_error: 0.150676                                            
[18]	valid_0's binary_error: 0.148403                                            
[19]	valid_0's binary_error: 0.147604                                            
[20]	valid_0's binary_error: 0.147113                                            
[21]	valid_0's binary_error: 0.145823                                            
[22]	valid_0's binary_error: 0.144595                                            
[23]	valid_0's binary_error: 0.142936                                            
[24]	valid_0's binary_error: 0.140848                                            
[25]	valid_0's binary_error: 0.140786                                            
[26]	valid_0's binary_error: 0.140356                                            
[27]	valid_0's binary_error: 0.140479                                            
[28]	valid_0's binary_error: 0.139619                                            
[29]	valid_0's b

[11]	valid_0's binary_error: 0.236241                                             
[12]	valid_0's binary_error: 0.236241                                             
[13]	valid_0's binary_error: 0.236241                                             
[14]	valid_0's binary_error: 0.236241                                             
[15]	valid_0's binary_error: 0.236241                                             
[16]	valid_0's binary_error: 0.236241                                             
[17]	valid_0's binary_error: 0.236241                                             
[18]	valid_0's binary_error: 0.236241                                             
[19]	valid_0's binary_error: 0.236241                                             
[20]	valid_0's binary_error: 0.236241                                             
[21]	valid_0's binary_error: 0.236241                                             
[22]	valid_0's binary_error: 0.236241                                             
[23]

[5]	valid_0's binary_error: 0.236241                                              
[6]	valid_0's binary_error: 0.236241                                              
[7]	valid_0's binary_error: 0.236241                                              
[8]	valid_0's binary_error: 0.236241                                              
[9]	valid_0's binary_error: 0.236241                                              
[10]	valid_0's binary_error: 0.236241                                             
[11]	valid_0's binary_error: 0.236241                                             
[12]	valid_0's binary_error: 0.236241                                             
[13]	valid_0's binary_error: 0.236241                                             
[14]	valid_0's binary_error: 0.236241                                             
[15]	valid_0's binary_error: 0.236241                                             
[16]	valid_0's binary_error: 0.236241                                             
[17]

...

[38]	valid_0's binary_error: 0.148342                                              
[39]	valid_0's binary_error: 0.148894                                              
[40]	valid_0's binary_error: 0.150307                                              
[41]	valid_0's binary_error: 0.151351                                              
[42]	valid_0's binary_error: 0.150676                                              
[43]	valid_0's binary_error: 0.150123                                              
[44]	valid_0's binary_error: 0.150246                                              
[45]	valid_0's binary_error: 0.149079                                              
[46]	valid_0's binary_error: 0.149509                                              
[47]	valid_0's binary_error: 0.149754                                              
[48]	valid_0's binary_error: 0.149693                                              
[49]	valid_0's binary_error: 0.149693                                       

[32]	valid_0's binary_error: 0.137961                                              
[33]	valid_0's binary_error: 0.138206                                              
[34]	valid_0's binary_error: 0.138391                                              
[35]	valid_0's binary_error: 0.139189                                              
[36]	valid_0's binary_error: 0.139926                                              
[37]	valid_0's binary_error: 0.140233                                              
[38]	valid_0's binary_error: 0.140295                                              
[39]	valid_0's binary_error: 0.140111                                              
[40]	valid_0's binary_error: 0.140602                                              
[41]	valid_0's binary_error: 0.140172                                              
[42]	valid_0's binary_error: 0.141339                                              
[43]	valid_0's binary_error: 0.140848                                       

[25]	valid_0's binary_error: 0.134951                                              
[26]	valid_0's binary_error: 0.134337                                              
[27]	valid_0's binary_error: 0.134521                                              
[28]	valid_0's binary_error: 0.133292                                              
[29]	valid_0's binary_error: 0.13317                                               
[30]	valid_0's binary_error: 0.13231                                               
[31]	valid_0's binary_error: 0.132002                                              
[32]	valid_0's binary_error: 0.131695                                              
[33]	valid_0's binary_error: 0.131511                                              
[34]	valid_0's binary_error: 0.131143                                              
[35]	valid_0's binary_error: 0.13059                                               
[36]	valid_0's binary_error: 0.130651                                       

[19]	valid_0's binary_error: 0.127211                                              
[20]	valid_0's binary_error: 0.127334                                              
[21]	valid_0's binary_error: 0.126658                                              
[22]	valid_0's binary_error: 0.127273                                              
[23]	valid_0's binary_error: 0.127457                                              
[24]	valid_0's binary_error: 0.127334                                              
[25]	valid_0's binary_error: 0.127334                                              
[26]	valid_0's binary_error: 0.127334                                              
[27]	valid_0's binary_error: 0.12801                                               
[28]	valid_0's binary_error: 0.127887                                              
[29]	valid_0's binary_error: 0.12758                                               
[30]	valid_0's binary_error: 0.127826                                       

[13]	valid_0's binary_error: 0.236241                                              
[14]	valid_0's binary_error: 0.236241                                              
[15]	valid_0's binary_error: 0.236241                                              
[16]	valid_0's binary_error: 0.236241                                              
[17]	valid_0's binary_error: 0.230037                                              
[18]	valid_0's binary_error: 0.229975                                              
[19]	valid_0's binary_error: 0.230037                                              
[20]	valid_0's binary_error: 0.208538                                              
[21]	valid_0's binary_error: 0.20086                                               
[22]	valid_0's binary_error: 0.190172                                              
[23]	valid_0's binary_error: 0.188882                                              
[24]	valid_0's binary_error: 0.187899                                       

In [5]:
print(best_val)
print(space_eval(spc, best))

0.12383292383292384
{'bagging_fraction': 0.646055407419704, 'feature_fraction': 0.8543972819051316, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.09836151876466383, 'metric': 'binary_error', 'min_data_in_leaf': 23.0, 'min_sum_hessian_in_leaf': 0.004036994353968463, 'num_leaves': 12.0, 'objective': 'binary'}
